In [30]:
import numpy as np
import pandas as pd

## 1.1 Описание данных.

TRAIN.CSV

* row_id: (int64) ID код строки.
* timestamp: (int64) время в миллисекундах между данным взаимодействием пользователя и первым событием завершения для этого пользователя.
* user_id: (int32) ID код пользователя.
* content_id: (int16) ID код контента, с которым взаимодействовал пользователь.
* content_type_id: (int8) 0 — если событие представляет собой вопрос, который задается пользователю, 1 — если событие представляет собой просмотр пользователем лекции.
* task_container_id: (int16) ID код пакета вопросов или лекций. Например, пользователь может увидеть три вопроса подряд перед тем, как увидеть объяснения к ним. Эти три вопроса будут иметь один и тот же task_container_id.
* user_answer: (int8) ответ пользователя на вопрос, если он есть. Значение -1 означает, что пользователь не отвечал (для лекций).
* answered_correctly: (int8) правильный ответ пользователя. Значение -1 означает, что это лекция, а не вопрос.
* prior_question_elapsed_time: (float32) среднее время в миллисекундах, которое пользователю потребовалось для ответа на каждый вопрос в предыдущем пакете вопросов, исключая лекции между ними. Поле пусто для первого пакета вопросов или лекций у пользователя. Обратите внимание, что это среднее время решения вопроса в предыдущем пакете.
* prior_question_had_explanation: (bool) показывалось ли пользователю объяснение и правильные ответы после предыдущего пакета вопросов, исключая лекции между ними. Значение одинаково для всех вопросов в пакете и пусто для первого пакета вопросов или лекций у пользователя. Обычно первые несколько вопросов являются частью диагностического теста для новичков, где пользователи не получают обратную связь.

QUESTIONS.CSV: метаданные для вопросов, задаваемых пользователям

* question_id: внешний ключ для столбца content_id в train/test, когда тип контента — это вопрос (content_type_id = 0).
* bundle_id: код, обозначающий, какие вопросы были показаны вместе.
* correct_answer: правильный ответ на вопрос. Можно сравнивать с ответами пользователей (user_answer) в train для проверки правильности ответа.
* part: релевантный раздел теста TOEIC.
* tags: один или несколько кодов-тегов для вопроса. Значение тегов не указано, но эти коды позволяют кластеризовать вопросы.

LECTURES.CSV: метаданные для лекций, просматриваемых пользователями в процессе обучения

* lecture_id: внешний ключ для столбца content_id в train/test, когда тип контента — это лекция (content_type_id = 1).
* part: код категории верхнего уровня для лекции.
* tag: один код-тег для лекции. Значение тегов не указано, но они могут быть использованы для кластеризации лекций.
* type_of: краткое описание основной цели лекции.


### Загрузка данных

Данные можно скачать из https://www.kaggle.com/competitions/riiid-test-answer-prediction/data

In [2]:
# Локальный путь до файлов
paths = ['./train.csv', './lectures.csv', './questions.csv']

In [3]:
df_train = pd.read_csv(paths[0])
df_lectures = pd.read_csv(paths[1])
df_questions = pd.read_csv(paths[2])

### "Первый взгляд" на данные

In [4]:
print('Train Data Preview:')
print(df_train.head())
print('\nLectures Data Preview:')
print(df_lectures.head())
print('\nQuestions Data Preview:')
print(df_questions.head())

# Общая информация о каждом наборе данных
print('\nTrain Data Info:')
print(df_train.info())

print('\nLectures Data Info:')
print(df_lectures.info())

print('\nQuestions Data Info:')
print(df_questions.info())

Train Data Preview:
   row_id  timestamp  user_id  content_id  content_type_id  task_container_id  \
0       0          0      115        5692                0                  1   
1       1      56943      115        5716                0                  2   
2       2     118363      115         128                0                  0   
3       3     131167      115        7860                0                  3   
4       4     137965      115        7922                0                  4   

   user_answer  answered_correctly  prior_question_elapsed_time  \
0            3                   1                          NaN   
1            2                   1                      37000.0   
2            0                   1                      55000.0   
3            0                   1                      19000.0   
4            1                   1                      11000.0   

  prior_question_had_explanation  
0                            NaN  
1                   

In [5]:
# Проверка на наличие пропущенных значений в каждом наборе данных
print('\nПропущенные значений в Train Data:')
print(df_train.isnull().sum())

print('\nПропущенные значений в Lectures Data:')
print(df_lectures.isnull().sum())

print('\nПропущенные значений в Questions Data:')
print(df_questions.isnull().sum())


Пропущенные значений в Train Data:
row_id                                  0
timestamp                               0
user_id                                 0
content_id                              0
content_type_id                         0
task_container_id                       0
user_answer                             0
answered_correctly                      0
prior_question_elapsed_time       2351538
prior_question_had_explanation     392506
dtype: int64

Пропущенные значений в Lectures Data:
lecture_id    0
tag           0
part          0
type_of       0
dtype: int64

Пропущенные значений в Questions Data:
question_id       0
bundle_id         0
correct_answer    0
part              0
tags              1
dtype: int64


In [6]:
# Базовая статистика для числовых данных в каждом наборе
print('\nБазовая статистика для Train Data:')
print(df_train.describe())

print('\nБазовая статистика для Lectures:')
print(df_lectures.describe())

print('\nБазовая статистика для Questions Data:')
print(df_questions.describe())


Базовая статистика для Train Data:
             row_id     timestamp       user_id    content_id  \
count  1.012303e+08  1.012303e+08  1.012303e+08  1.012303e+08   
mean   5.061517e+07  7.703644e+09  1.076732e+09  5.219605e+03   
std    2.922268e+07  1.159266e+10  6.197163e+08  3.866359e+03   
min    0.000000e+00  0.000000e+00  1.150000e+02  0.000000e+00   
25%    2.530758e+07  5.243436e+08  5.408116e+08  2.063000e+03   
50%    5.061517e+07  2.674234e+09  1.071781e+09  5.026000e+03   
75%    7.592275e+07  9.924551e+09  1.615742e+09  7.425000e+03   
max    1.012303e+08  8.742577e+10  2.147483e+09  3.273600e+04   

       content_type_id  task_container_id   user_answer  answered_correctly  \
count     1.012303e+08       1.012303e+08  1.012303e+08        1.012303e+08   
mean      1.935222e-02       9.040624e+02  1.376123e+00        6.251644e-01   
std       1.377596e-01       1.358302e+03  1.192896e+00        5.225307e-01   
min       0.000000e+00       0.000000e+00 -1.000000e+00       

### Выводы по 1.1.

Все данные имеют соответствующий тип, здесь обработка не нужна.


В Train Data есть значительные пропуски в столбцах prior_question_elapsed_time и prior_question_had_explanation, но в описании данных указано что пропуски это тоже информация (для первого пакета), поэтому эти пропуски не надо обрабатывать заполнением:
 * prior_question_elapsed_time: отсутствуют значения примерно в 2.35 млн строк.
 * prior_question_had_explanation: отсутствуют значения в около 392 тыс. строк.

В Lectures Data пропусков нет.

В Questions Data всего одно пропущенное значение в столбце tags.

## Обработка пропущенных значений

В столбце tags в Questions Data можно попробовать заполнить отсутствующее значение пустым списком или специальным кодом.

In [7]:
# Обработка пропущенных значений в df_questions
df_questions['tags'].fillna('unknown', inplace=True)

In [8]:
# Проверка после обработки пропусков
print('\nПропущенные значения в Questions Data после заполнения:')
print(df_questions.isnull().sum())


Пропущенные значения в Questions Data после заполнения:
question_id       0
bundle_id         0
correct_answer    0
part              0
tags              0
dtype: int64


## 1.2 Исследование признаков.

### 1.2.1 Студенты и взаимодействия.

In [9]:
# Общее количество студентов и количество их взаимодействий
num_users = df_train['user_id'].nunique()
num_interactions = len(df_train)
print(f"Количество уникальных студентов: {num_users}")
print(f"Количество всех взаимодействий: {num_interactions}")

# Среднее количество взаимодействий на одного студента
avg_interactions_per_user = num_interactions / num_users
print(f"Среднее количество взаимодействий на одного студента: {avg_interactions_per_user:.2f}")

# Топ-10 самых активных студентов по количеству взаимодействий
top_active_students = df_train['user_id'].value_counts().head(10)
print("Топ-10 самых активных студентов (ID и количество взаимодействий):")
print(top_active_students)

# Статистики по количеству взаимодействий: медиана, мода
median_interactions = df_train['user_id'].value_counts().median()
mode_interactions = df_train['user_id'].value_counts().mode()[0]
print(f"Медиана количества взаимодействий на одного студента: {median_interactions}")
print(f"Мода количества взаимодействий на одного студента: {mode_interactions}")

Количество уникальных студентов: 393656
Количество всех взаимодействий: 101230332
Среднее количество взаимодействий на одного студента: 257.15
Топ-10 самых активных студентов (ID и количество взаимодействий):
801103753     17917
1478712595    16914
1842816145    16851
455973631     16789
1660941992    16777
1743444187    16654
2146130037    16384
1047202059    16348
1615528747    16146
338684437     15963
Name: user_id, dtype: int64
Медиана количества взаимодействий на одного студента: 41.0
Мода количества взаимодействий на одного студента: 30


Сравнивая среднее и и медиану, можно построить гипотезу, что есть оооочень активные студенты.
Чаще всего кол-во взаимодействий это 30.

### 1.2.2 Лекции и вопросы.

In [10]:
# Количество лекций и вопросов
total_questions = df_questions.shape[0]
total_lectures = df_lectures.shape[0]
print(f"Количество вопросов: {total_questions}")
print(f"Количество лекций: {total_lectures}")

# Среднее количество вопросов на лекцию
avg_questions_per_lecture = total_questions / total_lectures
print(f"Среднее количество вопросов на лекцию: {avg_questions_per_lecture:.2f}")

# Топ-10 лекций, на которых задают больше всего и меньше всего вопросов
lecture_question_counts = df_train[df_train['content_type_id'] == 0].merge(df_lectures, left_on='content_id', right_on='lecture_id')['content_id'].value_counts()
most_questions_lectures = lecture_question_counts.head(10)
fewest_questions_lectures = lecture_question_counts.tail(10)
print("Топ-10 лекций, на которых задают больше всего вопросов (ID и количество):")
print(most_questions_lectures)
print("Топ-10 лекций, на которых задают меньше всего вопросов (ID и количество):")
print(fewest_questions_lectures)

# Статистики по количеству вопросов на лекцию: медиана, мода
median_questions_per_lecture = lecture_question_counts.median()
mode_questions_per_lecture = lecture_question_counts.mode()[0]
print(f"Медиана количества вопросов на лекцию: {median_questions_per_lecture}")
print(f"Мода количества вопросов на лекцию: {mode_questions_per_lecture}")

Количество вопросов: 13523
Количество лекций: 418
Среднее количество вопросов на лекцию: 32.35
Топ-10 лекций, на которых задают больше всего вопросов (ID и количество):
10688    126998
3852      51908
185       47047
335       44284
5588      32047
5694      28447
5266      27477
10075     26285
4100      26271
4425      24529
Name: content_id, dtype: int64
Топ-10 лекций, на которых задают меньше всего вопросов (ID и количество):
12811    174
12638    172
12831    167
12655    158
2476     144
12942    138
1308     135
13138    106
1907      71
4652      38
Name: content_id, dtype: int64
Медиана количества вопросов на лекцию: 4739.0
Мода количества вопросов на лекцию: 4600


Отсюда можно построить гипотезу, что есть прямо супер популярная лекция, это id 10688, возможно это базовая лекция для всех студентов, или же какая-нибудь форма на сайте, которую многим необходимо заполнять.

Тут среднее и медиана с модой отличаются в противоположную сторону, в сравнении с предыдущим пунктом, это может означать что "выбросы" (отклонения) от медианного значения очень большие.

### 1.2.3 Ответы.

In [11]:
# 3. Распределение правильных ответов
answered_correctly_counts = df_train['answered_correctly'].value_counts()
print("Распределение правильных и неправильных ответов студентов:")
print(answered_correctly_counts)

Распределение правильных и неправильных ответов студентов:
 1    65244627
 0    34026673
-1     1959032
Name: answered_correctly, dtype: int64


Студенты почти в 2 раза больше отвечали правильно чем не правильно)) студенты молодцы. Необходимо исследовать среднее значения или группировать, чтобы понять каков тренд для разных групп.

-1 - это лекции.

### 1.2.4 Время на решение вопросов.

In [12]:
# Анализ времени на решение вопросов
elapsed_time_stats = df_train['prior_question_elapsed_time'].describe()
print("Статистика времени на ответы студентов:")
print(elapsed_time_stats)

Статистика времени на ответы студентов:
count    9.887879e+07
mean     2.542381e+04
std      1.994815e+04
min      0.000000e+00
25%      1.600000e+04
50%      2.100000e+04
75%      2.966600e+04
max      3.000000e+05
Name: prior_question_elapsed_time, dtype: float64


Необходимо учесть что статистики даны в миллисекундах.

Среднее время ответа составляет около 25 секунд, а медианое время 21с, что указывает на относительно симметричное распределение времени, при этом стандартное отклонение в 19.9с показывает что, возможно, вопросы бывают разной сложности, либо разный уровень подготовки студентов. 

Минимальное значение равное 0с возможно связано с ошибками сбора данных, либо ошибку работы сайта, т.к. физически невозможно за 0.0с ответить на вопрос, даже случайно. Максимальное значение равное 5 минутам (300 000 миллисекунд) показывают что некоторые вопросы вызывают затрудения, либо содержат информацию для прочтения\расчета. На вопросы с этой части необходимо обратить внимание, чтобы качественно повысить уровень данных и пользовательский опыт. 

Первый квартиль (25%) и третий квартиль (75%) показывают, что 50% студентов отвечают на вопросы в диапазоне от 16с до 29.6с. Это говорит о том, что большинство студентов тратят около 16-30 секунд на ответ.

### 1.2.5 Вопросы и объяснения.

In [13]:
# Влияние объяснений на правильность ответа
explanation_group = df_train.pivot_table(values='answered_correctly', index='prior_question_had_explanation', aggfunc='mean')
print("Средняя правильность ответа в зависимости от объяснений:")
print(explanation_group)

Средняя правильность ответа в зависимости от объяснений:
                                answered_correctly
prior_question_had_explanation                    
False                                     0.237025
True                                      0.673189


Здесь мы видим качественную разницу для вопросов, на который был получен правильный ответ в зависимости от объяснения.

### 1.2.6 Популярные лекции и вопросы.

In [14]:
# Анализ повторяющихся лекций и вопросов
# Какие лекции смотрят чаще всего
most_watched_lectures = df_train[df_train['content_type_id'] == 1]['content_id'].value_counts().head(10)
print("Наиболее популярные лекции (ID и количество просмотров):")
print(most_watched_lectures)

Наиболее популярные лекции (ID и количество просмотров):
3153     37024
21411    27051
10540    24312
29695    24126
15888    22627
27845    21250
26335    21118
32312    20943
29544    20941
18545    19167
Name: content_id, dtype: int64


In [15]:
# Какие вопросы чаще всего задают
most_asked_questions = df_train[df_train['content_type_id'] == 0]['content_id'].value_counts().head(10)
print("Наиболее часто задаваемые вопросы (ID и количество):")
print(most_asked_questions)

Наиболее часто задаваемые вопросы (ID и количество):
6116    213605
6173    202106
4120    199372
175     195861
7876    190170
7900    180858
2063    176043
2065    176043
2064    176043
4492    173769
Name: content_id, dtype: int64


### 1.2.7 Стастистика по студентам

In [16]:
# Анализ студентов, кто дал больше правильных ответов
student_correctness = df_train.groupby('user_id')['answered_correctly'].mean().describe()
print("Статистика средней правильности ответов студентов:")
print(student_correctness)

Статистика средней правильности ответов студентов:
count    393656.000000
mean          0.530048
std           0.160218
min          -0.142857
25%           0.421627
50%           0.550000
75%           0.647059
max           1.000000
Name: answered_correctly, dtype: float64


Средняя правильность ответов составляет около 53%, медиана составляет 55%, что немного выше среднего значения, что указывает на небольшую асимметрию в распределении — больше студентов имеют успеваемость выше среднего, а значения близкие к 50% что указывает на то, что в целом студенты правильно отвечают на чуть больше половины вопросов. Это может означать умеренный уровень сложности тестов или нехватку знаний у некоторых студентов.

Минимальная правильность равна -0.142857, что явно некорректное значение, вероятно, обусловленное ошибками в данных или неправильной обработкой. Значение правильности не может быть отрицательным, и это может указывать на необходимость очистки данных.

Максимальная правильность равна 100%, что означает, что есть студенты, которые правильно ответили на все вопросы. Это может указывать на наличие (задротов) очень хорошо подготовленных студентов.

Первый квартиль (25%) составляет 42%, а третий квартиль (75%) — 64.7%, что говорит о том, что половина студентов имеют правильность ответов в диапазоне от 42% до 65%. Это диапазон средней успеваемости.

### 1.2.8 Наборы вопросов.

In [17]:
# Анализ распределения взаимодействий по task_container_id
task_container_distribution = df_train['task_container_id'].value_counts().head(10)
print("Наиболее часто используемые task_container_id (и количество взаимодействий):")
print(task_container_distribution)

Наиболее часто используемые task_container_id (и количество взаимодействий):
14    804285
15    798539
4     692079
5     690051
6     688813
7     684275
11    403521
10    400660
8     400019
9     399641
Name: task_container_id, dtype: int64


Видим что первые 6 task_container_id (набор вопросов) намного популярнее остальных, возможно это так же базовые какие-либо вопросы.

### 1.2.9 Посещаемость и успеваемость.

In [18]:
# Влияние количества лекций на успеваемость студентов
lecture_counts = df_train[df_train['content_type_id'] == 1].groupby('user_id').size()
correctness_vs_lectures = df_train[df_train['content_type_id'] == 0].groupby('user_id')['answered_correctly'].mean().reset_index()
correctness_vs_lectures = correctness_vs_lectures.merge(lecture_counts.rename('num_lectures'), on='user_id', how='left').fillna(0)
print("Корреляция между количеством лекций и успеваемостью студентов:")
print(correctness_vs_lectures.corr())

Корреляция между количеством лекций и успеваемостью студентов:
                     user_id  answered_correctly  num_lectures
user_id             1.000000           -0.001259     -0.000145
answered_correctly -0.001259            1.000000      0.197191
num_lectures       -0.000145            0.197191      1.000000


Корреляция между количеством лекций и успеваемостью студентов составляет около 0.197, что является положительной, но слабой корреляцией. Это может означать, что студенты, которые посещают больше лекций, имеют некоторую тенденцию к лучшей успеваемости, однако этот эффект не очень значителен. Это может быть связано с тем, что просто количество посещенных лекций недостаточно для значительного улучшения успеваемости — важны также качество и понимание материала. Если мы говорим про линейную прямую связь, возможно связь сложнее и простой линейной корреляции тут не достаточно.

Для этого необходимо дополнительное исследование.

In [22]:
# Разделение студентов на группы по количеству посещенных лекций и сравнение средней правильности ответов
lecture_bins = [-1, 5, 15, 50, np.inf]
lecture_labels = ['Мало лекций', 'Среднее количество', 'Много лекций', 'Очень много лекций']
correctness_vs_lectures['lecture_group'] = pd.cut(correctness_vs_lectures['num_lectures'], bins=lecture_bins, labels=lecture_labels)
grouped_correctness = correctness_vs_lectures.groupby('lecture_group')['answered_correctly'].mean()
print("Средняя правильность ответов в зависимости от количества посещенных лекций:")
print(grouped_correctness)

Средняя правильность ответов в зависимости от количества посещенных лекций:
lecture_group
Мало лекций           0.525751
Среднее количество    0.635502
Много лекций          0.648186
Очень много лекций    0.662682
Name: answered_correctly, dtype: float64


1. Студенты, посещающие больше лекций, показывают лучшую успеваемость. Средняя правильность ответов студентов с минимальным количеством лекций составляет около 52.6%, в то время как у студентов, посещающих очень много лекций, этот показатель достигает 66.3%.
2. Можно сделать вывод, что большее количество лекций положительно влияет на понимание материала и успеваемость студентов.

In [23]:
# Распределение количества лекций среди студентов с разным уровнем успеваемости
correctness_bins = [-1, 0.3, 0.7, 1]
correctness_labels = ['Низкая успеваемость', 'Средняя успеваемость', 'Высокая успеваемость']
correctness_vs_lectures['correctness_group'] = pd.cut(correctness_vs_lectures['answered_correctly'], bins=correctness_bins, labels=correctness_labels)
lecture_distribution = correctness_vs_lectures.groupby('correctness_group')['num_lectures'].describe()
print("Распределение количества лекций среди студентов с разным уровнем успеваемости:")
print(lecture_distribution)

Распределение количества лекций среди студентов с разным уровнем успеваемости:
                         count      mean        std  min  25%  50%  75%    max
correctness_group                                                             
Низкая успеваемость    40113.0  0.246977   4.139014  0.0  0.0  0.0  0.0  250.0
Средняя успеваемость  286238.0  4.658438  15.163006  0.0  0.0  0.0  2.0  386.0
Высокая успеваемость   67305.0  9.147953  21.715244  0.0  0.0  1.0  7.0  397.0


1. Студенты с низкой успеваемостью в среднем посещают значительно меньше лекций (среднее количество лекций около 0.25).
2. Студенты с высокой успеваемостью посещают больше лекций (в среднем около 9), что подтверждает положительное влияние посещения лекций на успеваемость.

In [24]:
# Выявление лекций, которые оказывают наибольшее влияние на успеваемость
lecture_effectiveness = df_train[df_train['content_type_id'] == 1].merge(df_lectures, left_on='content_id', right_on='lecture_id')
lecture_effectiveness = lecture_effectiveness.merge(correctness_vs_lectures[['user_id', 'answered_correctly']], on='user_id', how='left')
lecture_effectiveness_group = lecture_effectiveness.groupby('lecture_id')['answered_correctly_y'].mean().sort_values(ascending=False).head(10)
print("Лекции, которые оказывают наибольшее влияние на успеваемость (топ-10):")
print(lecture_effectiveness_group)

Лекции, которые оказывают наибольшее влияние на успеваемость (топ-10):
lecture_id
6220     0.771250
32604    0.731384
8240     0.720630
18903    0.720299
30194    0.713750
12509    0.699107
27402    0.698856
25004    0.697984
8461     0.695722
9831     0.694240
Name: answered_correctly_y, dtype: float64


1. Лекции с ID из списка демонстрируют наибольшее влияние на успеваемость студентов, с показателем правильности ответов выше 70%.
2. Эти лекции могут быть особенно эффективными и стоит изучить их содержание для определения ключевых факторов успеха.

### 1.2.10 Последний раздел теста TOEIC.

In [17]:
# Анализ характеристик вопросов и лекций по признаку 'part'
questions_part_distribution = df_questions['part'].value_counts()
lectures_part_distribution = df_lectures['part'].value_counts()
print("Распределение вопросов по частям:")
print(questions_part_distribution)
print("Распределение лекций по частям:")
print(lectures_part_distribution)

Распределение вопросов по частям:
5    5511
2    1647
3    1562
4    1439
6    1212
7    1160
1     992
Name: part, dtype: int64
Распределение лекций по частям:
5    143
6     83
2     56
1     54
7     32
4     31
3     19
Name: part, dtype: int64


Неясно какие отсюда можно сделать выводы, кроме тогоч то в в части 5 больше вопросов и больше лекций.

In [26]:
# Анализ связи между частями лекций и вопросов
questions_lectures_merged = df_questions.merge(df_lectures, on='part', how='inner', suffixes=('_question', '_lecture'))
part_relationship = questions_lectures_merged['part'].value_counts()
print("Связь между частями вопросов и лекций:")
print(part_relationship)

Связь между частями вопросов и лекций:
5    788073
6    100596
2     92232
1     53568
4     44609
7     37120
3     29678
Name: part, dtype: int64


1. Часть 5 содержит наибольшее количество вопросов и лекций, что может указывать на особое внимание к этой части в учебном процессе.
2. Часть 6 и 2 также имеет большое количество вопросов и лекций, что может свидетельствовать о её важности в курсе.
3. Остальные части имеют свзь одинакового порядка.

### 1.2.11 Сложность тем.

In [28]:
# Использование информации о тегах лекций и вопросов для анализа, какие темы сложнее всего
tags_difficulty = df_train[df_train['content_type_id'] == 0].merge(df_questions, left_on='content_id', right_on='question_id')
print("Средняя правильность ответов по тегам вопросов:")
print(tags_difficulty.groupby('tags')['answered_correctly'].mean())

Средняя правильность ответов по тегам вопросов:
tags
1                       0.607333
1 162                   0.622810
10 111 92               0.837507
10 164 102              0.761476
10 164 162 29           0.879346
                          ...   
98 97 35 117 122 162    0.718192
98 97 35 146 122        0.716996
98 97 46 135            0.684939
98 97 77 135 162        0.888320
unknown                 1.000000
Name: answered_correctly, Length: 1520, dtype: float64


Здесь анализ затруднен, т.к. tags скорее всего необходимо обработать через get_dummies или ohe, но это не входит в рамки данного проекта.

### 1.2.12 Сложность вопросов.

In [29]:
# Анализ успешности ответов по 'bundle_id' для оценки сложности конкретных пакетов вопросов
bundle_difficulty = tags_difficulty.groupby('bundle_id')['answered_correctly'].mean()
print("Средняя правильность ответов по bundle_id:")
print(bundle_difficulty)

# Топ-10 лучших и худших пакетов вопросов по средней правильности ответов
top_bundles = bundle_difficulty.sort_values(ascending=False).head(10)
bottom_bundles = bundle_difficulty.sort_values(ascending=True).head(10)
print("Топ-10 лучших пакетов вопросов по средней правильности ответов:")
print(top_bundles)
print("Топ-10 худших пакетов вопросов по средней правильности ответов:")
print(bottom_bundles)

# Связь успешности пакетов вопросов с другими параметрами
bundle_analysis = tags_difficulty.merge(bundle_difficulty.rename('bundle_mean_correctness'), on='bundle_id')
print("Корреляция между средней правильностью ответов по пакетам и другими признаками:")
print(bundle_analysis.corr())

Средняя правильность ответов по bundle_id:
bundle_id
0        0.907721
1        0.890646
2        0.554281
3        0.779437
4        0.613215
           ...   
13518    0.786866
13519    0.571429
13520    0.672474
13521    0.808046
13522    0.913953
Name: answered_correctly, Length: 9765, dtype: float64
Топ-10 лучших пакетов вопросов по средней правильности ответов:
bundle_id
12923    1.000000
12806    1.000000
10033    1.000000
4741     1.000000
12615    1.000000
5823     1.000000
12809    1.000000
3572     1.000000
12745    0.994595
12803    0.994413
Name: answered_correctly, dtype: float64
Топ-10 худших пакетов вопросов по средней правильности ответов:
bundle_id
1484     0.000000
9220     0.144753
11231    0.172831
8981     0.182177
9547     0.183336
8697     0.183818
8986     0.197763
3878     0.199648
13439    0.200647
8351     0.203025
Name: answered_correctly, dtype: float64
Корреляция между средней правильностью ответов по пакетам и другими признаками:


C:\Users\User\AppData\Local\Temp\ipykernel_18876\3745067142.py:17: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(bundle_analysis.corr())


                               row_id  timestamp   user_id  content_id  \
row_id                       1.000000  -0.000006  0.999990    0.001104   
timestamp                   -0.000006   1.000000 -0.000037    0.062655   
user_id                      0.999990  -0.000037  1.000000    0.001106   
content_id                   0.001104   0.062655  0.001106    1.000000   
content_type_id                   NaN        NaN       NaN         NaN   
task_container_id            0.001448   0.434853  0.001454    0.064442   
user_answer                  0.000121   0.001048  0.000120    0.011568   
answered_correctly          -0.000414   0.029045 -0.000418   -0.020850   
prior_question_elapsed_time  0.000294   0.013934  0.000308    0.151661   
question_id                  0.001104   0.062655  0.001106    1.000000   
bundle_id                    0.001104   0.062654  0.001106    1.000000   
correct_answer               0.000230  -0.001087  0.000230    0.017385   
part                         0.001891 

Топ-10 лучших пакетов вопросов по средней правильности ответов имеют очень высокую правильность, вплоть до 100%. Это может говорить о том, что эти пакеты содержат относительно простые вопросы или что материал, покрываемый этими пакетами, был хорошо усвоен.

Топ-10 худших пакетов имеют низкую среднюю правильность (до 20%), что может свидетельствовать о сложности тем, представленных в этих пакетах, или недостаточной подготовке студентов.

Наиболее сильная положительная корреляция наблюдается между bundle_mean_correctness и answered_correctly (коэффициент корреляции 0.323), что указывает на прямую связь между сложностью пакета и успешностью ответов студентов.

С другой стороны, отрицательная корреляция между bundle_mean_correctness и part (-0.232) может говорить о том, что определенные части учебного курса сложнее других, что сказывается на успешности выполнения вопросов.

### Выводы по 1.2.

Анализ данных по взаимодействиям студентов с образовательной платформой показал, что существует целый ряд факторов, которые влияют на успеваемость студентов. Студенты, которые чаще посещают лекции, демонстрируют более высокие результаты на тестах, что подтверждает значимость и ценность лекционного материала. Несмотря на это, корреляция между количеством лекций и успеваемостью оказалась умеренной, что указывает на наличие других важных факторов, влияющих на успеваемость, таких как качество обучения и личная мотивация студентов.

Сравнительный анализ лекций и вопросов выявил, что некоторые лекции и вопросы значительно более популярны, что может быть связано с их обязательностью для всех студентов или важностью тем, которые они покрывают. Были также обнаружены аномалии в данных, такие как минимальное время ответа, равное нулю, и отрицательные значения правильности, что требует дальнейшей очистки данных для повышения точности и достоверности анализа.

При анализе сложности пакетов вопросов было установлено, что существуют как простые, так и сложные наборы вопросов. Топ-10 пакетов с наивысшей правильностью ответов показывают, что эти вопросы либо были хорошо усвоены студентами, либо являлись менее сложными. С другой стороны, худшие пакеты указывают на необходимость дополнительного внимания к сложным темам или на пробелы в подготовке студентов.

Важной частью анализа также стало выявление связей между сложностью тем и правильностью ответов. Определенные части курса оказались сложнее других, что отразилось на успеваемости студентов. Эти выводы позволяют выдвинуть гипотезу о необходимости адаптации учебных материалов и внедрения более персонализированного подхода к обучению.

Эти результаты помогут лучше понять потребности студентов и улучшить образовательный процесс, адаптируя сложность вопросов и лекций в соответствии с их подготовкой и текущим уровнем знаний.

# Общий вывод.

Данный анализ является минимально достаточным для понимания общих тенденций и трендов в данном датасете.

Проект был реализован используя только библиотеку pandas (по условию), различные библиотеки по построению графика не были использованы, хотя это могло упростить работы, но при этом, датасеты довольно большие (больше 7.5+ GB), что затрудняло обработку данных графиками.